**Jacob Lustig-Yaeger**  
**ASTR 555 - Planetary Atmospheres**  
**Prof. David Catling**  

*Preamble--* The following homework submission is the output file from an iPython Notebook and contains the original code written and run for the calculations in this assignment. Including the code does make this a lengthy document (perhaps unnecessarily lengthy), but I think it's a pretty nifty way to provide my answers and show my work. 

In [56]:
# Import numpy for math
import numpy as np

# Import units to keep me honest
import astropy.units as u

## Problem 1:
>[16 points total] In the future, planetary astronomers discover an inhabited extrasolar planet, which they call “Houston2”. This planet has a foul and noxious polluted atmosphere, which the media speculates is caused by incessant, unregulated industrial activity. However, all that is really known is that the atmosphere is rich in hydrocarbons, that the temperature at the 200-mbar level in the atmosphere is 273.16 K, and that the surface temperature is 546.32 K. The astronomers all agree that the atmosphere has an essentially adiabatic structure. But there are three camps regarding the atmospheric composition: (i) one group believes that the atmosphere is 50% by volume Xe and 50% methane. (ii) the other group believes that the atmosphere is pure lead tetraethyl Pb(C2H5)4 --- the unwelcome substance found in leaded gasoline (iii) a final group argue for a composition of 50% He and 50% methane.

>(a) What is the surface atmospheric pressure on “Houston2” in each model (i) through (iii)? [12 pts]  
>(b) Explain how atmospheres (i) and (iii) will differ in their vertical extent? You can assume that Houston2 has the same gravity as the Earth, and a useful thing to do would be to compare heights at the 0.2 bar level. [4 pts]  

>[In answering part (a), you will need heat capacity. Go and look at a thermal physics textbook. You’ll find that each degree of freedom of a molecule or atom contributes R/2 to Cv, the molar heat capacity at constant volume. For a molecule of N atoms, there are 3N degrees of freedom. For example, for He there are 3N = 3 degrees of freedom corresponding to 3 translational modes in the x, y and z directions, so that Cv = 3R/2. For CO, for example, there are 3N = 6 degrees of freedom corresponding to 3 translational modes, 1 vibrational mode and 2 rotational modes. But whether certain modes are excited depends on temperature. H2, for example, has Cv = 3R/2 at 40K, Cv ~ 5R/2 at 300K and CV = 6R/2 at 2000K and approaches 7R/2 in the limit (H2 dissociates before it gets there). Each vibrational mode has a kinetic part associated with movement of atoms and a potential part like spring energy in a simple harmonic oscillator, which can constitute two dofs. So the maximum possible Cv = (3 trans. + 2 rots + 2 vibs)R/2 = 7R/2 for a diatomic molecule. At room temperature, vibrational modes tend to be inactive, so Cv = 5R/2 for most diatomic molecules. For the purpose of this homework, ignore these complications and assume that the 3N rule holds. Recall also that molar heat capacity at constant pressure is related to Cv via the equation Cp = Cv + R. (Data: Masses in grams/mole: Xe = 130, C = 12, H = 1, He = 4, Pb = 206)].  

Let's start by cataloging the given quantities:  

In [61]:
T200 = 273.16 * u.K; print T200
P200 = 200.0 * 1e-3 * u.bar; print P200
Tsurf = 546.32 * u.K; print Tsurf

273.16 K
0.2 bar
546.32 K


Define gram/mol unit

In [59]:
amu = u.gram / u.mol; amu

Unit("g / mol")

The masses of atoms in grams/mol are:

In [ ]:
mmw_Xe = 130. * amu
mmw_C = 12. * amu
mmw_H = 1.0 *amu
mmw_He = 4.0 * amu
mmw_Pb = 206. * amu

The masses of the molecules are:

In [63]:
# Mass of methane
mmw_CH4 = mmw_C + 4*mmw_H; mmw_CH4

<Quantity 16.0 g / mol>

In [64]:
# Mass of Lead Tetraethyl
mmw_PbTet = mmw_Pb + 4*(2*mmw_C+5*mmw_H); mmw_PbTet

<Quantity 322.0 g / mol>

Now each case has a different composition, given in terms of volume mixing ratio.

In [66]:
# Cases
#(i)
fXe = 0.5
fCH4 = 0.5
#(ii)
fPbTet = 1.0
#(iii)
fHe = 0.5
fCH4 = 0.5

### Part a)  
From the notes,  
$$ T = T_{ref} \left ( \frac{P}{P_{ref}} \right ) ^{\frac{\gamma - 1}{\gamma}} $$  
where $\gamma$ is the ratio of specific heats ($\gamma = c_p/c_v$).
Solving for P,  
$$ P = P_{ref} \left ( \frac{T}{T_{ref}} \right ) ^{\frac{\gamma}{\gamma - 1}} $$
which can be used to find the surface pressure using the P,T known at 200 mbar and the surface temperature.
We can use kinetic theory to relate $\gamma$ to the number of degrees of freedom available to a gas particle via the following relation,  
$$ \gamma = 1 + \frac{2}{N_{dof}} .$$ 
Finally, using the simplifying assumption that $N_{dof} = 3N$, where $N$ is the number of atoms, we get the following relation:  
$$ P = P_{ref} \left ( \frac{T}{T_{ref}} \right ) ^{1+\frac{N_{dof}}{2}} $$  

Let's write the above equation as a function so we can call it multiple times.

In [65]:
def P1(T, Pr, Tr, Ndof):
    """
    Calculate the pressure assuming a known reference temperature and pressure, a deeper temperature, 
    and assuming a dry adiabat.
    
    Parameters
    ----------
    T : Temperature at which to determine the pressure
    Pr : Reference pressure
    Tr : Reference temperature
    Ndof : Gas number of degrees of freedom
    
    Returns
    -------
    P1 : Pressure at given T
    """
    return Pr * (T / Tr)**(1.+Ndof/2.)

We will determine the number of degrees of freedom in a mixed gas by weighting the individual gas constituents by their respective volume mixing ratios. 

In [67]:
# Set the number of atoms for each gas
N_Xe = 1.
N_CH4 = 5.
N_PbT = 29.
N_He = 1.0

# Compute the Number of degrees of freedom for each case
Ndof1 = 3. * (fXe*N_Xe + fCH4*N_CH4)
Ndof2 = 3. * (fPbTet*N_PbT)
Ndof3 = 3. * (fHe*N_He + fCH4*N_CH4)

# print intermediate values for checking math
print "N_dof:", Ndof1, Ndof2, Ndof3
print "gamma:", (1.+2./Ndof1), (1.+2./Ndof2), (1.+2./Ndof3)
print "exponent:", 1  + Ndof1/2., 1  + Ndof2/2., 1  + Ndof3/2.

N_dof: 9.0 87.0 9.0
gamma: 1.22222222222 1.02298850575 1.22222222222
exponent: 5.5 44.5 5.5


Given a different number of degrees of freedom for each possible atmosphere, we calculate the surface pressures given below:

In [7]:
Pi = P1(Tsurf, P200, T200, Ndof1)
print "Surface pressure for case (i) is", Pi

Surface pressure for case (i) is 9.05096679919 bar


In [8]:
Pii = P1(Tsurf, P200, T200, Ndof2)
print "Surface pressure for case (ii) is", Pii

Surface pressure for case (ii) is 4.97582161916e+12 bar


In [9]:
Piii = P1(Tsurf, P200, T200, Ndof3)
print "Surface pressure for case (iii) is", Piii

Surface pressure for case (iii) is 9.05096679919 bar


### Part b)  

The atmospheric pressure scale height is  
$$ H = \frac{k \bar{T}}{\bar{m} g } $$

Define the Boltzmann Constant,

In [10]:
k = 1.38064852e-23 * (u.m**2 *u.kg * u.second**-2 * u.K**-1 * u.kg.in_units(u.g) * u.g/u.kg); k

<Quantity 1.3806485199999997e-20 g m2 / (K s2)>

and the pressure scale height.

In [11]:
def pressure_scale_height(T, m, g, k=k):
    """
    """
    return (k * T) / (m * g)

Assuming the gravity is the same as that on Earth

In [12]:
g = 9.8 * u.m / u.s**2; g

<Quantity 9.8 m / s2>

Set Avagadro's Number:

In [13]:
NA = 6.022140857e23 * u.mol**-1; NA

<Quantity 6.022140857e+23 1 / mol>

Calculate the mean molecular weight for each possible case:

In [14]:
mu1 = (fXe * mmw_Xe + fCH4 * mmw_CH4); mu1

<Quantity 73.0 g / mol>

In [15]:
mu2 = (fPbTet * mmw_PbTet); mu2

<Quantity 322.0 g / mol>

In [16]:
mu3 = (fHe * mmw_He + fCH4 * mmw_CH4); mu3

<Quantity 10.0 g / mol>

Calculate the mean atmospheric mass for each possible case:

In [17]:
m1 = mu1 / NA; m1

<Quantity 1.2121934995118297e-22 g>

In [18]:
m2 = mu2 / NA; m2

<Quantity 5.346935710175468e-22 g>

In [19]:
m3 = mu3 / NA; m3

<Quantity 1.660539040427164e-23 g>

We'll calculate the pressure scale height at the 200 mbar level. The pressure scale height is only valid in isothermal regions of an atmosphere. However, 200 mbar is probably close to the tropopause (particularly because the temperature here is at zero degrees celsius perhaps hinting that water will freeze out around this level, contributing latent heat and cold trapping the water below). Using the temperature at this level to calculate the pressure scale height will give a reasonable approximation of the extent of the atmosphere *above* 200 mbar. 

In [20]:
H1 = pressure_scale_height(T200, m1, g) * u.m.in_units(u.km) * u.km/u.m; H1

<Quantity 3.17469647155898 km>

In [21]:
H2 = pressure_scale_height(T200, m2, g) * u.m.in_units(u.km) * u.km/u.m; H2

<Quantity 0.7197293242975326 km>

In [22]:
H3 = pressure_scale_height(T200, m3, g) * u.m.in_units(u.km) * u.km/u.m; H3

<Quantity 23.175284242380553 km>

The pressure scale height gives us an estimate of the atmospheric extent. Since we know the temperature at 200 mbar and the gravity, then the scale height for the 3 possible cases is determined soley by the atmospheric mass. The pure lead tetraethyl atmosphere, which has a mean molecular weight of 322 g/mol, is the heaviest and therefore has the smallest atmospheric extent with a scale height of 0.72 km. On the other hand, the 50% Helium, 50% methane atmosphere has the lowest mass ($\mu = 10$ g/mol) and therefore the largest atmospheric extent ($H = 23.17$ km). The 50% Xe, 50% methane atmosphere has an intermediate extent ($H = 3.17$ km).

## Problem 2:  
>[4 points total] A spherical hot-air balloon is designed to float at the ~1 bar, 60 K level in the atmosphere of Neptune. Scientists hope to track the winds in the atmosphere of Neptune by following the motion of the balloon. The balloon is also instrumented with atmospheric sensors. The weight of the balloon and payload is 30 kg. What balloon volume and radius is required for a “hot air” temperature of 100 K?
(Data: mean molar mass of Neptune atmosphere = 2.6 g/mol). [4 pts]  

Here's what we know:

In [23]:
P2 = 1.0 * u.bar; P2

<Quantity 1.0 bar>

In [24]:
T2 = 60. * u.K; T2

<Quantity 60.0 K>

In [25]:
m_payload = 30.0 * u.kg; m_payload 

<Quantity 30.0 kg>

In [26]:
T_hot_air = 100.0 * u.K; T_hot_air

<Quantity 100.0 K>

In [27]:
mu_Nep = 2.6 * u.g/u.mol; mu_Nep

<Quantity 2.6 g / mol>

An object inbedded in another medium will float due to buoyancy if the downward weight ($mg$) of the object is equal to the weight of the displaced medium. This is Archimedes' principle. In other words, the net force on the payload must be zero. From the notes we can write the buoyancy force as,  
$$ F_{buoy} = -g (\rho_{in} - \rho_{out}) V $$  
which must balance the downward force of the balloon payload,  
$$ F_{pay} = m_p g .$$  
Setting $F_{buoy} = F_{pay}$ and solving for the balloon volume, we get  
$$V = \frac{m_p}{\rho_{out} - \rho_{in}}$$

Ultimately it is the temperature difference between the gas inside the balloon versus outside that drives the density difference that provides the buoyant force. We'll calculate the density of an ideal gas:
$$ \rho = \frac{\bar{m} P }{k T} $$  
Inside the balloon the density is:

In [28]:
rho_in = ((mu_Nep / NA) * P2.decompose()) / (k * T_hot_air); rho_in 

<Quantity 0.3127082267911769 kg / m3>

and outside the balloon the density is:

In [29]:
rho_out = ((mu_Nep / NA) * P2.decompose()) / (k * T2); rho_out

<Quantity 0.5211803779852948 kg / m3>

Now we'll solve for the balloon volume, 

In [30]:
V = m_payload / (rho_out - rho_in); V

<Quantity 143.90411298661004 m3>

And, assuming the balloon is spherical we can trivially find the radius: 
$$ V = \frac{4}{3} \pi R^3 \rightarrow R = \left ( \frac{3V}{4 \pi} \right ) ^{1/3}$$

In [31]:
R = ((3. * V)/(4. * np.pi))**(1./3.); R

<Quantity 3.2508345429120604 m>

A 3.25-m radius balloon seems reasonably sized. 

## Problem 3:  

>[6 points total] In deriving the hydrostatic equation for an isothermal planetary atmosphere, we assumed that the scale height $H$ was much smaller than the radius of the planet; that is, that $g$ can be considered a constant throughout the atmosphere, as a reasonable approximation. Suppose early Titan had much hydrogen in its atmosphere. If so, our approximation is no longer valid because $g = (GM)/r^2$ at planetocentric distance $r$.  

>(i) Re-derive the hydrostatic equation for this case, taking into account the variation of g with planetocentric distance. I suggest you start with $dP / P = −(\bar{m}g / R\bar{T} )dr$ .
[3 pts]  
>(ii) Assume that early Titan’s atmosphere was made of pure hydrogen (H$_2$). What would be the scale height using the usual formula and assuming an isothermal temperature of 90 K? Look up Titan’s radius. As a percentage, how much is the scale height a fraction of Titan’s radius? [Titan $g = 1.37$ m/s$^2$] [2 pts]  
>(iii) Why is a pure hydrogen atmosphere unlikely to be found around a planet like Titan? [1 pt]   

### Part 1:  
Starting with the hydrostatic equation,  
$$ \frac{dP}{P} = - \left ( \frac{\bar{m}g}{R\bar{T}} \right) dr $$  
and plugging in $$ g = \frac{GM}{r^2} $$ we get,  
$$ \frac{dP}{P} = - \left ( \frac{GM\bar{m}}{R\bar{T}} \right ) \frac{dr}{r^2}$$  
Now we want to integrate both sides of the above expression from the surface (at $z=R_p$) to some height z,  
$$ \int_{p_s}^{p(z)} \frac{dP}{P} = - \left ( \frac{GM\bar{m}}{R\bar{T}} \right ) \int_{R_p}^{z} \frac{dr}{r^2}$$  
$$ \ln \left ( \frac{p(z)}{p_s} \right ) = \left ( \frac{GM\bar{m}}{R\bar{T}} \right ) \left ( \frac{1}{z} - \frac{1}{R_p} \right ) $$  
Exponentiate both sides and simplifying our result,  
$$ p(z) = p_s \exp \left ( \frac{-GM\bar{m}}{R_p R\bar{T}} \right ) \exp \left ( \frac{GM\bar{m}}{z R\bar{T}}  \right )
$$  
If we define the surface gravity,  
$$ g_s = \frac{GM}{R_p^2} $$  
Then we can write the typical pressure scale height as  
$$ H = \frac{R \bar{T}}{g_s} $$  
Our solution now simplifies to: 
$$ p(z) = p_s \exp \left ( \frac{-R_p}{H} \right ) \exp \left ( \frac{R_p^2}{H z}  \right ) $$  

### Part 2:   
We'll start with the isothermal temperature of

In [32]:
T3 = 90 * u.K; T3

<Quantity 90.0 K>

The mean molecular weight for hydrogen gas:

In [33]:
muH2 = 2.0 * u.g / u.mol; muH2

<Quantity 2.0 g / mol>

which, in grams, is

In [34]:
mH2 = muH2 / NA; mH2

<Quantity 3.3210780808543285e-24 g>

The gravity of Titan:

In [35]:
g_titan = 1.37 * u.m / u.second**2; g_titan

<Quantity 1.37 m / s2>

We calculate the pressure scale height to be:

In [36]:
H3 = pressure_scale_height(T3, mH2, g_titan) * u.m.in_units(u.km) * u.km/u.m; H3

<Quantity 273.10269617896796 km>

Titan's radius is 

In [37]:
R_titan = 2576. * u.km; R_titan

<Quantity 2576.0 km>

This means that if Titan ever had a pure hydrogen atmosphere, the pressure scale height would have been 

In [38]:
H3 / R_titan * 100 * u.percent

<Quantity 10.60181273986677 %>

the radius of Titan.

### Part 3:  
A pure hydrogen atmosphere is unlikely to be found for a planet like Titan because the low mass of hydrogen leads to such a vertically extended atmosphere that the molecules can easily escape the low gravity of the small world. Becuase temperature is the average kinetic energy of the molecules in a gas, low mass molecules achieve much higher velocities than heavier molecules (like carbon dioxide and hydrocarbons) at any given temperature. Atmospheric escape can occur when the velocity of molecules exceeds the escape velocity of a planet. This condition is easily met for hydrogen in the atmospheres of small planets and moons. 

## Problem 4:  
>[7 points] An Earth-based spectroscopic measurement of the strength of some features in the infrared spectrum of Mars reports a column abundance of CO2 of 8000 cm-amagat.  

>a. What is the corresponding surface pressure of carbon dioxide in millibars? [Data: molar mass of CO2 is 44 g/mol; Martian gravity g = 3.72 m s-2].
[4 pts]  

>b. The paper reporting this result does not bother to tell us whether the abundance refers to the sub-Earth region of Mars (i.e. a vertical incidence region of Mars in the direct line-of-sight from Earth) or to an average over the entire disk of Mars. Would the surface pressure be less, the same, or greater than the value calculated in (a) if the reported abundance is actually a whole- disk average? Justify your answer in a qualitative fashion. (With a bit of thought and ‘one step’ arithmetic, it is possible to calculate disk-average pressure versus line of sight pressure. You’re looking to deduce a weighting factor for a disk average rather than a direct path perpendicular to the surface at the equator. This gets a bonus point, but if you’re not sure, don’t sweat it because it's subtle).
[2 pts + 1 bonus pt]

### Part 1:  
Let's define the atm unit of pressure in terms of bars:

In [43]:
atm = 1.01325 * u.bar; atm

<Quantity 1.01325 bar>

Then the column abundance thing ($\mathcal{N}$) is

In [44]:
N = 8000. * u.cm* atm ;N

<Quantity 8106.0 bar cm>

The mean molecular weight of CO$_2$ is

In [45]:
muCO2 = 44. * u.g / u.mol; muCO2

<Quantity 44.0 g / mol>

which is

In [46]:
mCO2 = muCO2 / NA; mCO2

<Quantity 7.306371777879523e-23 g>

Finall, the gravity of Mars and the Loschmidt number are 

In [47]:
g_mars = 3.72 * u.m / u.second**2; g_mars

<Quantity 3.72 m / s2>

In [48]:
n0 = 2.687e19 * u.cm**-3; n0

<Quantity 2.687e+19 1 / cm3>

From the notes, we can calculate pressure from the column abundance, atmospheric mass, and gravity using this relation  
$$ p(z) = N_c(z) \bar{m} g $$  
where true column abundance can be computed from via,  
$$ N_c(z) = n_0 \mathcal{N} $$  
This gives the following in fundamental SI unit bases:

In [50]:
p_mars1 = N.decompose() * n0.decompose() * mCO2.decompose() * g_mars.decompose(); p_mars1

<Quantity 59199627.33545207 kg2 / (m2 s4)>

Note that the fundamental SI units of pressure are $\frac{\text{kg}}{\text{m s}^2}$, e.g. 

In [188]:
u.bar.decompose()

Unit("100000 kg / (m s2)")

We have calculated something that has units of pressure squared! Therefore, it would seems that this equation from the notes:  

$$ p(z) = N_c(z) \bar{m} g $$ 

should actually be:  

$$ p(z) = \sqrt{N_c(z) \bar{m} g} $$  

Using this expression we get:

In [53]:
p_mars2 = np.sqrt((n0 * N).decompose() * g_mars.decompose() * mCO2.decompose()); p_mars2

<Quantity 7694.129407246286 kg / (m s2)>

Converting to bars, this is

In [55]:
p_mars2_bar = (p_mars2 / u.bar.decompose()).decompose() * u.bar; p_mars2_bar

<Quantity 0.07694129407246286 bar>

This is about 77 mbar of CO$_2$ at the surface of Mars, roughly an order of magnitude higher than the true surface pressure of Mars (${\sim}6$ mbar). 

### Part 2:  
If the reported column abundance were disk-integrated, rather than the vertical column at the sub-Earth point, the inferred surface pressure would be lower. The sub-Earth path to the surface is the minimum possible path over the ensemble of paths that connect the observer and all visible locations of the surface of Mars. Integrating the column abundance over the entire disk will necessarily include contributions from these longer paths, which encounter more CO$_2$ molecules, and bias us towards higher surface pressures if we misinterpret it for only the sub-observer view.  

Since the true surface pressure of the CO$_2$-dominated Martian atmosphere is ${\sim}6$ mbar, we are off by a factor of about 10. So perhaps the weighting factor is 10 or, better yet, $\pi^2$, since that is the area of the disk in normalized units. 

To rigorously solve for the difference between inferred surface pressures using the nadir and disk-integrated views we must calculate the disk-averaged path length to the surface. This requires some geometry and a 2D integral over the visible disk of Mars. The answer is probably $\pi^2$.  